[Drippypale](https://github.com/drippypale/ROSALIND)  
Email: drippypale@gmail.com  

Problem: **[Implement AdditivePhylogeny](https://rosalind.info/problems/ba7c/)**

In [1]:
import numpy as np

In [2]:
with open('ba7c.in', 'r') as f:
    lines = f.read().splitlines()
    n = int(lines[0])
    dist_matrix = list()

    for l in lines[1:]:
        dist_matrix.extend([int(x) for x in l.split()])
    
    dist_matrix = np.array(dist_matrix, dtype=np.int64).reshape((n, n))

First we need to find the $\delta$ value which we're going to reduce all the hanging edges by it. The `limb_length()` function will return the $\delta$ value:

In [3]:
def limb_length(dist_matrix, n, j):
    min_length = np.inf

    for i in range(n):
        for k in range(n):
            l = (dist_matrix[i][j] + dist_matrix[j][k] - dist_matrix[i][k])/2
            if k != i and k != j and i != j and l < min_length:
                min_length = l
    return int(min_length)

Next step is to find a *degenerative* tirple $i, n, k$ which satisfies: $D_{in} + D_{nk} = D_{nk}$ so that the node $n$, can be removed:

In [4]:
import itertools

def find_degenerative_triple(dist_matrix):
    n = dist_matrix.shape[0]
    cn_list = itertools.combinations(range(n), 2)
    for i, k in cn_list:
        if dist_matrix[i, k] == dist_matrix[i, n - 1] + dist_matrix[n - 1, k]:
            return i, k
    # for k in range(dist_matrix.shape[0]-1):
    #     arr = dist_matrix[k] - dist_matrix[-1]
    #     index = np.where(arr == dist_matrix[k, -1])
    #     if len(index[0]) > 0:
    #         return (index[0][0], k)
    # return None

Also we need to save the $X = D_{in}$ value for reconstructing the tree.

After removing the $n-th$ column and row, and constructing the tree for the reduced matrix(by calling the `additive_phylogeny()` for the reduced matrix which we will see in a second), we will try to find the proper spot to insert (if needed) the removed node. By proper spot I mean ***a spot on the $i-k$ path, and with a $distance = X$ from the $i$ leaf***.    

So wee need to find the **$i-k$** path first, and then find that spot we already talked:

In [5]:
from queue import LifoQueue

'''
Given the Tree(T) and the nodes i and k,
returns the path from i to k.
'''
def dfs_find_path(T, i, k):
    marked = list()
    stack = LifoQueue()

    stack.put([i]) # the stack contains the path to the node + node itself

    while not stack.empty():
        pi = stack.get() 
        x = pi[-1]
        marked.append(x)
        if x == k:
            return pi
        t = [_ for _ in T[x].keys() if not _ in marked]
        for c in t:
            stack.put(pi + [c])


As we discussed, the `find_x()` will return the spot to inset the removed node:

In [6]:
'''
Given the Tree(T), and the distance x, and the path i-k,
this will return the closest a, b which a distance = x, lies in between them (this is the spot for the removed node).
also returns the (x - dist[a, i]) and the ()
'''
def find_x(T, x, path):
    dist = 0
    for k in range(1, len(path)):
        a, b = path[k - 1], path[k]
        if dist + T[a][b] > x:
            return a, b, x - dist, (dist + T[a][b]) - x
        dist += T[a][b]

Now it's time to combine the above steps, all in a recursive function called `additive_phylogeny()`:

In [7]:
import itertools

next_node_index = n # holds the index of the next inner node

def additive_phylogeny(dist_matrix, n):

    if n == 2: # if it's only 2 nodes, just connect them with an edge, and return the tree
        T = {
            0: {
                1: dist_matrix[0][1]
                },
            1: {
                0: dist_matrix[1][0]
                }
        }
        return T
    
    # 1. reduce all the hanging edges by limb_length
    delta = limb_length(dist_matrix, n, n - 1)
    dist_matrix[:-1, -1] -= delta
    dist_matrix[-1, :-1] -= delta
    
    # 2. find the degenerate triple i, n, k
    i, k = find_degenerative_triple(dist_matrix)
    # 2.1 store the D[i, n] to reconstruct the tree
    x = dist_matrix[i, n - 1]

    # 3. remove the n-th column and row. and build the tree for the reduced matrix
    T: dict[dict] = additive_phylogeny(dist_matrix[:n - 1, :n - 1], n - 1)

    # 4. find the spot of the removed node to place it back
    # 4.1 find the path between i-k first
    i_k__path = dfs_find_path(T, i, k)

    # 4.2 find the proper spot on the path:
    i_a, b_k, i__x, x__k = find_x(T, x, i_k__path) # the path is smt like: i---a-------b---k
    new_node = i_a

    # 5. check if we need to insert a new node for the removed one
    if i__x != 0:
        global next_node_index

        new_node = next_node_index
        next_node_index += 1

        # remove edges between a and b
        del T[i_a][b_k]
        del T[b_k][i_a]

        # insert new_node between a and b with the distance i__x from the a, and x__k from the b
        T[new_node] = dict()
        
        T[i_a][new_node] = i__x
        T[new_node][i_a] = i__x

        T[b_k][new_node] = x__k
        T[new_node][b_k] = x__k

    # 6. add leaf n back to T by creating a limb (v, n) of length limbLength
    T[n - 1] = dict()
    
    T[new_node][n - 1] = delta
    T[n - 1][new_node] = delta
    return T
    
    

In [8]:
T = additive_phylogeny(dist_matrix, n)

In [9]:
with open('ba7c.out', 'w') as f:
    for i in sorted(T.keys()):
            for j in sorted(T[i].keys()):
                f.write(f'{i}->{j}:{T[i][j]}\n')